* 1.0 About Author:-

  - 1.1 Description : Complete Auto EDA Analysis with multiple Excel sheet

  - 1.2 Date of Submission: 25 July 2025

  - 1.3 Author Name: Ahasan U Haque

  - 1.4 Email: ahasan7990@gmail.com

  - 1.5 kaggle_id: ansarianam

* 2.0 About Dataset

  - 2.1- Description: This dataset is downloaded from Keggal , link https://www.kaggle.com/datasets/vivekkumarkamat/vendor-performance-analysis-project
  

  - 2.2- Context :

While many public datasets (on Kaggle and the like) provide for EDA Store data, there are not many counterpart datasets available for Store anywhere on the web.

  - 2.3- Content:
Each Table (row) has values for Store salesprice, Inovicedate and many more in multiple sheets.

  - 2.4- Acknowledgements:
This information is scraped from the Kaggle dataset. This Store information would not be available without it.

  - 2.5- Inspiration: Auto EDA and Correlation Analysis Notebook
    - This notebook performs the following tasks:
    - Loads and cleans multiple CSV files
    - Performs exploratory data analysis (EDA)
    - Detects and handles missing values and outliers
    - Correlates across tables
    - Exports results to Excel and Word reports
  
  

* 3.0-Task:-

    - 3.1- We have to do Explorartory Data Analysis(EDA) on Store dataset. The tasks involved in this EDA include data cleaning to handle missing or inconsistent data, data visualization to identify patterns and outliers and statistical analysis to understand relationships between variables. We will also focus on feature engineering to extract meaningful information for further analysis.

   - 3.2- Objective:- The primary objective of this Exploratory Data Analysis (EDA) on the  Store dataset is to understand the underlying patterns, trends, and relationships between different variables. This will help us gain insights into revenue gain for the store.

* 4.0- Importing Liberaries :

    We will useing following libraries to complete our Analysis.

    - Pandas: Provides data structures and functions needed for manipulating structured data.
    - NumPy: Used for numerical computing, such as arrays and linear algebra.
    - Matplotlib: Allows creation of static, animated, and interactive visualizations in Python.
    - Seaborn: Based on Matplotlib, it provides a high-level interface for drawing attractive    statistical graphics.
    - SciPy: Used for scientific computing and technical computing.
    - sklearn: Provides a range of supervised and unsupervised learning algorithms via a consistent interface in Python. It’s used for data mining, data analysis and machine learning.

In [1]:
# Part 1: Configuration and Imports
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import openpyxl
from docx import Document
from docx.shared import Inches
import matplotlib

warnings.filterwarnings("ignore")
matplotlib.use('Agg')

c:\Users\AHASANUL HAQUE\.conda\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Part 2: Define Paths and Parameters
DATA_DIR = r"D:/Auto EDR/Data"
EDA_OUTPUT_DIR = "eda_outputs"
MASTER_CSV_PATH = os.path.join(EDA_OUTPUT_DIR, "master_data.csv")
MERGE_LIMIT = 800

os.makedirs(EDA_OUTPUT_DIR, exist_ok=True)

In [3]:
# Part 3: Read Limited Rows from Each CSV
def read_csv_limited(filepath, nrows=5000):
    try:
        df = pd.read_csv(filepath, nrows=nrows, encoding='utf-8', delimiter=',', on_bad_lines='skip')
        print(f"Loaded {filepath} ({len(df)} rows)")
        return df
    except Exception as e:
        print(f"Failed loading {filepath}: {e}")
        return pd.DataFrame()

files = {
    "sales": "Sales.csv",
    "purchases": "Purchases.csv",
    "vendor_invoice": "vendor_invoice.csv",
    "purchase_prices": "purchase_prices.csv",
    "begin_inventory": "begin_inventory.csv",
    "end_inventory": "end_inventory.csv",
}

dataframes = {name: read_csv_limited(os.path.join(DATA_DIR, fname)) for name, fname in files.items()}

Loaded D:/Auto EDR/Data\Sales.csv (5000 rows)
Loaded D:/Auto EDR/Data\Purchases.csv (5000 rows)
Loaded D:/Auto EDR/Data\vendor_invoice.csv (5000 rows)
Loaded D:/Auto EDR/Data\purchase_prices.csv (5000 rows)
Loaded D:/Auto EDR/Data\begin_inventory.csv (5000 rows)
Loaded D:/Auto EDR/Data\end_inventory.csv (5000 rows)


In [4]:
#  Part 4: Clean DataFrame Columns and Drop All-Null Rows
def clean_data(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    df.dropna(how='all', inplace=True)
    return df

dataframes = {name: clean_data(df) for name, df in dataframes.items()}

In [5]:
#  Part 5: Detect and Handle Outliers (IQR Method)
def remove_outliers_iqr(df):
    df_clean = df.copy()
    numeric_cols = df_clean.select_dtypes(include=np.number)
    Q1 = numeric_cols.quantile(0.25)
    Q3 = numeric_cols.quantile(0.75)
    IQR = Q3 - Q1
    is_outlier = ((numeric_cols < (Q1 - 1.5 * IQR)) | (numeric_cols > (Q3 + 1.5 * IQR)))
    df_clean[is_outlier] = np.nan
    return df_clean

dataframes = {name: remove_outliers_iqr(df) for name, df in dataframes.items()}

In [6]:
# Part 5.1: Clean and Handle Null Values Across All DataFrames

def handle_nulls(df, strategy="auto"):
    """
    Handles missing values in a DataFrame.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        strategy (str): Strategy for imputation: "auto", "mean", "median", "mode", or "drop".
    
    Returns:
        pd.DataFrame: Cleaned DataFrame with handled nulls.
    """
    df = df.copy()
    
    for col in df.columns:
        if df[col].isnull().sum() == 0:
            continue  # No nulls in this column
        
        if strategy == "drop":
            df = df.dropna(subset=[col])
        
        elif strategy == "mean" and pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(df[col].mean())
        
        elif strategy == "median" and pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].fillna(df[col].median())
        
        elif strategy == "mode":
            df[col] = df[col].fillna(df[col].mode()[0])
        
        elif strategy == "auto":
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col] = df[col].fillna(df[col].median())
            else:
                df[col] = df[col].fillna("Unknown")
        
        else:
            df[col] = df[col].fillna("Unknown")
    
    return df

# Apply null handling to all dataframes
dataframes_cleaned = {}
for name, df in dataframes.items():
    cleaned_df = handle_nulls(df, strategy="auto")
    dataframes_cleaned[name] = cleaned_df
    print(f"✅ Nulls handled in '{name}'")

# Update the dataframes reference to cleaned version
dataframes = dataframes_cleaned



✅ Nulls handled in 'sales'
✅ Nulls handled in 'purchases'
✅ Nulls handled in 'vendor_invoice'
✅ Nulls handled in 'purchase_prices'
✅ Nulls handled in 'begin_inventory'
✅ Nulls handled in 'end_inventory'


In [7]:
# Part 6: Merge All CSVs into a Master File (max 5000 rows)

import os
import pandas as pd

MERGE_LIMIT = 5000
OUTPUT_DIR = "eda_outputs"
MASTER_CSV_PATH = os.path.join(OUTPUT_DIR, "master_data.csv")

def create_master_df(dataframes):
    common_keys = ["product_id", "item_id", "vendor_id"]
    keys_present = [key for key in common_keys if all(key in df.columns for df in dataframes.values())]
    
    dfs = list(dataframes.values())
    if not dfs:
        raise ValueError("No dataframes to merge.")
    
    master_df = dfs[0]
    for df in dfs[1:]:
        for key in keys_present:
            if key in df.columns and key in master_df.columns:
                master_df = pd.merge(master_df, df, on=key, how='outer')
                break  # merge once on first matching key
    
    return master_df.head(MERGE_LIMIT)

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Try writing the master CSV file
try:
    master_df = create_master_df(dataframes)
    master_df.to_csv(MASTER_CSV_PATH, index=False)
    print(f"✅ Master CSV written to: {MASTER_CSV_PATH}")
except PermissionError as e:
    print("❌ Unable to write the file. Please close the file if it's open elsewhere.")
    raise e


✅ Master CSV written to: eda_outputs\master_data.csv


In [8]:
# Part 7: Vendor Performance Analysis (Integrated into EDA Pipeline)

# Set a higher row limit for master merge
MERGE_LIMIT = 5000  # Increase from default 800 to 5000

def vendor_performance_analysis(master_df, output_path="eda_outputs/vendor_performance.xlsx"):
    """
    Analyze vendor performance based on combined sales and purchase data.

    Parameters:
    - master_df: DataFrame containing merged data from all input tables.
    - output_path: Path to save the vendor performance summary Excel file.
    """
    
    # Check for required column
    if "vendor_id" not in master_df.columns:
        print("❌ 'vendor_id' column not found. Skipping vendor analysis.")
        return

    # Create a working copy of the merged dataframe
    performance = master_df.copy()

    # Convert sales and purchase price columns to numeric (handling errors as NaN)
    for col in ["sales_price", "purchase_price"]:
        if col in performance.columns:
            performance[col] = pd.to_numeric(performance[col], errors='coerce')

    # Group data by vendor_id to calculate performance metrics
    vendor_stats = performance.groupby("vendor_id").agg(
        total_sales=pd.NamedAgg(column="sales_price", aggfunc="sum"),             # Total sales amount
        total_purchases=pd.NamedAgg(column="purchase_price", aggfunc="sum"),      # Total purchases amount
        avg_sales_price=pd.NamedAgg(column="sales_price", aggfunc="mean"),        # Average sales price
        avg_purchase_price=pd.NamedAgg(column="purchase_price", aggfunc="mean"),  # Average purchase price
        transaction_count=pd.NamedAgg(column="vendor_id", aggfunc="count")        # Total number of records per vendor
    ).fillna(0)  # Replace NaN with 0 for clean reporting

    # Calculate gross margin (sales - purchases)
    vendor_stats["gross_margin"] = vendor_stats["total_sales"] - vendor_stats["total_purchases"]

    # Save the output to an Excel file
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    vendor_stats.to_excel(output_path)

    print(f"✅ Vendor performance report saved at: {output_path}")


In [9]:
# Part 8: Generate EDA Reports for Each Table
def generate_eda_reports(dataframes):
    for name, df in dataframes.items():
        report = ProfileReport(df, title=f"EDA Report: {name}", explorative=True)
        report_path = os.path.join(EDA_OUTPUT_DIR, f"{name}_EDA.html")
        report.to_file(report_path)
        print(f"EDA report saved: {report_path}")

generate_eda_reports(dataframes)

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 81.01it/s]


EDA report saved: eda_outputs\sales_EDA.html


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 57.55it/s]


EDA report saved: eda_outputs\purchases_EDA.html


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 82.44it/s]


...

EDA report saved: eda_outputs\purchase_prices_EDA.html


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 154.11it/s]


EDA report saved: eda_outputs\begin_inventory_EDA.html


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 129.46it/s]

EDA report saved: eda_outputs\end_inventory_EDA.html


In [10]:
#  Part 8: Inter-Table Correlation Matrix
def find_inter_table_correlations(dataframes, output_excel_path):
    numeric_frames = [df.select_dtypes(include=np.number).add_prefix(f"{name}_") for name, df in dataframes.items()]
    merged = pd.concat(numeric_frames, axis=1).dropna(axis=1, how='all')
    correlation_matrix = merged.corr()
    correlation_matrix.to_excel(output_excel_path, sheet_name='Correlations')
    print(f"Correlation matrix saved to {output_excel_path}")

In [11]:
#  Part 9: Summary Observations per Table
def generate_observation_report(dataframes, output_excel_path):
    observations = []
    for name, df in dataframes.items():
        obs = {"Table": name}
        num_cols = df.select_dtypes(include='number')
        cat_cols = df.select_dtypes(include='object')

        obs["Rows"] = df.shape[0]
        obs["Columns"] = df.shape[1]
        obs["Missing %"] = round(df.isnull().sum().sum() / (df.shape[0]*df.shape[1]) * 100, 2)

        if df.isnull().sum().max() > 0:
            col = df.isnull().sum().idxmax()
            obs["Most Missing Column"] = f"{col} ({df[col].isnull().sum()} missing)"
        else:
            obs["Most Missing Column"] = "None"

        if not cat_cols.empty:
            top_col = cat_cols.nunique().idxmin()
            top_val = df[top_col].mode().iloc[0]
            freq = df[top_col].value_counts().iloc[0]
            obs["Most Frequent Value"] = f"{top_col}: '{top_val}' ({freq})"
        else:
            obs["Most Frequent Value"] = "N/A"

        if not num_cols.empty:
            corr = num_cols.corr().abs()
            corr.values[[range(len(corr))]*2] = 0
            pair = corr.stack().idxmax()
            obs["Top Correlation"] = f"{pair[0]} & {pair[1]}: {round(corr.max().max(), 2)}"
        else:
            obs["Top Correlation"] = "N/A"

        observations.append(obs)

    obs_df = pd.DataFrame(observations)
    with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        obs_df.to_excel(writer, sheet_name='Observations', index=False)
    print(f"Observation sheet saved in {output_excel_path}")

In [12]:
#  Part 10: Export to Word Document
def export_to_word(dataframes, output_path="summary_report.docx"):
    doc = Document()
    doc.add_heading("Auto EDA Summary Report", 0)

    for name, df in dataframes.items():
        doc.add_heading(f"{name.capitalize()} Table", level=1)
        doc.add_paragraph(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

        missing = df.isnull().sum()
        if missing.sum() > 0:
            most_missing = missing.idxmax()
            doc.add_paragraph(f"Most Missing Column: {most_missing} ({missing[most_missing]} values)")

        if not df.select_dtypes(include='object').empty:
            col = df.select_dtypes(include='object').nunique().idxmin()
            val = df[col].mode().iloc[0]
            freq = df[col].value_counts().iloc[0]
            doc.add_paragraph(f"Most Frequent Value: {col} = {val} ({freq}x)")

    doc.save(output_path)
    print(f"Word report saved to {output_path}")

In [14]:
#  Part 11: Run Full Pipeline
def full_pipeline():
    correlation_output = os.path.join(EDA_OUTPUT_DIR, "inter_table_correlations.xlsx")
    find_inter_table_correlations(dataframes, correlation_output)
    generate_observation_report(dataframes, correlation_output)
    export_to_word(dataframes)

full_pipeline()

Correlation matrix saved to eda_outputs\inter_table_correlations.xlsx
Observation sheet saved in eda_outputs\inter_table_correlations.xlsx
Word report saved to summary_report.docx
